<a href="https://colab.research.google.com/github/dgo2dance/qwen-tune/blob/main/LLaMA-Factory-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check GPU configuration
!nvidia-smi

Thu Jan 25 10:29:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。
##如記事本有出問題可在以下儲存庫回報
> https://github.com/ADT109119/LLaMA-Factory-Colab

In [ ]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib

!git clone https://gitee.com/duanshuyong/llama_factory.git
%cd ./llama_factory
!pip install kaleido
!pip install -r requirements.txt
!pip install bitsandbytes>=0.39.0

#用比較殘暴的方式開啟 Gradio 的分享連結
train_web_py_file_path = "./src/train_web.py"
try:
  with open(train_web_py_file_path, 'r') as file:
    file_content = file.read()
  modified_content = file_content.replace("share=False", "share=True")
  with open(train_web_py_file_path, 'w') as file:
    file.write(modified_content)
except Exception as e:
    print(f'ERROR: {str(e)}')

In [ ]:
! pip install cohere  openai tiktoken
! pip install transformers_stream_generator
! pip install transformers==4.36.2
! git lfs install
# !git clone https://huggingface.co/Qwen/Qwen-1_8B
!git clone https://huggingface.co/Qwen/Qwen-1_8B-Chat

In [5]:
#@title STEP 2 開啟WebUI
#@markdown #STEP 2
#@markdown ##開啟WebUI
#@markdown ##Run the WebUI
%cd /content/llama_factory

!CUDA_VISIBLE_DEVICES=0 python src/train_web.py

/content/llama_factory
2024-01-25 10:34:10.739593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 10:34:10.739701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 10:34:10.878262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 10:34:13.464626: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://f67ef6c9e84d92ee53.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Termi

In [ ]:
%cd /content/Qwen-1_8B-Chat
! ls

/content/Qwen-1_8B-Chat
assets				   generation_config.json	     qwen_generation_utils.py
cache_autogptq_cuda_256.cpp	   LICENSE			     qwen.tiktoken
cache_autogptq_cuda_kernel_256.cu  model-00001-of-00002.safetensors  README.md
config.json			   model-00002-of-00002.safetensors  tokenization_qwen.py
configuration_qwen.py		   modeling_qwen.py		     tokenizer_config.json
cpp_kernels.py			   model.safetensors.index.json
examples			   NOTICE


In [ ]:
! pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.26.1
aiofiles                         23.2.1
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
b

#STEP 3 上傳輸出的檔案到 HuggingFace


In [7]:
#@title STEP 3.1 安裝HuggingFace API
#@markdown #STEP 3.1
#@markdown ##為了上傳模型至HuggingFace，需要先安裝HF的函式庫
#@markdown ##In order to upload the model to HuggingFace, we have to install the HF library first.
!pip install huggingface_hub

In [8]:
#@title STEP 3.2 使用API Token登入
#@markdown #STEP 3.2
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login

from huggingface_hub import notebook_login

notebook_login()

In [10]:
#@title STEP 3.3 上傳模型
#@markdown #STEP 3.3
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login
from huggingface_hub import HfApi
api = HfApi()

model_id = "shuyong/qwen-1_8-fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shuyong/qwen-1_8-fine-tune-model/commit/eee3d6b5e05c983ec14aff3d318738ce0d4e4582', commit_message='Upload folder using huggingface_hub', commit_description='', oid='eee3d6b5e05c983ec14aff3d318738ce0d4e4582', pr_url=None, pr_revision=None, pr_num=None)

#STEP 4 保存到Google Drive(自選)

In [ ]:
#@title STEP 4.1 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 4.2 複製檔案到Google Drive
Google_Drive_Folder = "/content/drive/MyDrive/LLaMA-Factory/" #@param {type:"string"}

!cp -rf /content/LLaMA-Factory/saves {Google_Drive_Folder}